In [1]:
import joblib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch
import pandas as pd
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering

# Any results you write to the current directory are saved as output.

/kaggle/input/articles/AllArticles.csv
/kaggle/input/covidd/biorxiv_data.csv
/kaggle/input/covidd/comm_use_subset.csv
/kaggle/input/bertuncaselargesquad/vocab.txt
/kaggle/input/bertuncaselargesquad/config.json
/kaggle/input/bertuncaselargesquad/pytorch_model.bin


In [2]:
print(os.listdir('/kaggle/input/bertuncaselargesquad'))

['vocab.txt', 'config.json', 'pytorch_model.bin']


In [36]:
import torch
import pandas as pd
import numpy as np
from transformers import BartForConditionalGeneration, BartConfig, BartTokenizer

class Summary_Model(object):
    def __init__(self, summary_model_path, token_path, device):
        self.summary_model_path=summary_model_path
        self.token_model_path=token_path
        self.model=None
        self.tokenizer=None
        self.device=device
        self.create_models()

    def create_models(self):
        self.model = BartForConditionalGeneration.from_pretrained(self.summary_model_path)
        self.tokenizer = BartTokenizer.from_pretrained(self.token_model_path)
        self.model = self.model.to(self.device)

    def get_summary(self, data, count):
        ### Top confident answer to article
        total_abstract = ''
        for i in range(len(data[:count])):
            abss, ans = data.loc[i, ['answer_sent', 'Answer']].values
            total_abstract += (abss + ".")
        ARTICLE_TO_SUMMARIZE = total_abstract

        ### Token the article, if larger than 1024, then split the article
        tokens = self.tokenizer.tokenize(ARTICLE_TO_SUMMARIZE)
        max_seq_length = 1024
        longer = 0
        all_tokens = []
        if len(tokens) > 1024:
            for i in range(0, len(tokens), max_seq_length):
                tokens_a = tokens[i:i + max_seq_length]
                one_token = self.tokenizer.batch_encode_plus([tokens_a], max_length=1024, return_tensors='pt')
                all_tokens.append(one_token)

        Summary_text = []

        ## decode the model output as summary text
        def decode_text(sum_ids):
            text = ''
            for g in sum_ids:
                text = text + self.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False)
            return text

        ## Summary model
        model = self.model.to(self.device)
        model.eval()
        Summary_text = ''
        for inputs in all_tokens:
            summary_ids = model.generate(inputs['input_ids'].to(self.device), num_beams=2, max_length=1000,
                                         early_stopping=True)
            Summary_text = Summary_text + " " + decode_text(summary_ids)

        if Summary_text == '':
            Summary_text = "Can't find summary of answer"
        
        return Summary_text



In [37]:
import torch
import pandas as pd
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
from tqdm import notebook

class QuestionAnswering(object):
    def __init__(self, bert_model_path, token_model_path, device):
        self.bert_model_path=bert_model_path
        self.token_model_path=token_model_path
        self.model=None
        self.tokenizer=None
        self.device=device
        self.create_models()
        self.sum_model=Summary_Model(summary_model_path=Bart, token_path=Bart, device=device)


    def create_models(self):
        self.model= BertForQuestionAnswering.from_pretrained(self.bert_model_path)
        self.tokenizer=BertTokenizer.from_pretrained(self.token_model_path)
        self.model=self.model.to(self.device)
        
    def get_answer_sentence(self, text, answer):
        text=text.lower()
        answer=answer.lower()
        split_byans = text.split(answer)
        if len(split_byans)==1:
            return split_byans[0][split_byans[0].rfind(". ")+1:]+" "+answer
        else: 
            return split_byans[0][split_byans[0].rfind(". ")+1:]+" "+answer+split_byans[1][:split_byans[1].find(". ")+1]

    def create_answer_text(self, tokens, start=0, stop=-1):
        tokens = tokens[start: stop]
        if '[SEP]' in tokens:
            sepind = tokens.index('[SEP]')
            tokens = tokens[sepind+1:]
        txt = ' '.join(tokens)
        txt = txt.replace(' ##', '')
        txt = txt.replace('##', '')
        txt = txt.strip()
        txt = " ".join(txt.split())
        txt = txt.replace(' .', '.')
        txt = txt.replace('( ', '(')
        txt = txt.replace(' )', ')')
        txt = txt.replace(' - ', '-')
        txt_list = txt.split(' , ')
        txt = ''
        nTxtL = len(txt_list)
        if nTxtL == 1:
            return txt_list[0]
        newList =[]
        for i,t in enumerate(txt_list):
            if i < nTxtL -1:
                if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                    newList += [t,',']
                else:
                    newList += [t, ', ']
            else:
                newList += [t]
        return ''.join(newList)

    def train_model_func(self, question, document):
        token_ids = self.tokenizer.encode(question, document)
        tokens = self.tokenizer.convert_ids_to_tokens(token_ids)
        answers = []
        nWords = len(document.split())
        cs = []
        if len(token_ids) > 512:
            #divIndex = len(token_ids) // 2
            divIndex =int(np.ceil(nWords/2))
            all_sp = document.split()
            docs = [' '.join(all_sp[:divIndex]), ' '.join(all_sp[divIndex:])]
            token_ids = [self.tokenizer.encode(question, doc) for doc in docs]
        else:
            token_ids = [token_ids]

        for token_idp in token_ids:
            tokens = self.tokenizer.convert_ids_to_tokens(token_idp)
            sep_index = token_idp.index(self.tokenizer.sep_token_id)
            

            # Because of CLS
            questP = sep_index + 1
            secp = len(token_idp) - questP

            seq_types = [0] * questP + [1] * secp

            if len(seq_types)<512:

                start_index, end_index = self.model(torch.tensor([token_idp]).to(self.device),
                                                token_type_ids=torch.tensor([seq_types]) \
                                                .to(self.device))
            else:
                start_index, end_index = self.model(torch.tensor([token_idp[:512]]).to(self.device),
                                                    token_type_ids=torch.tensor([seq_types[:512]]) \
                                                    .to(self.device))
            start_index = start_index[:,1:-1]
            end_index = end_index[:,1:-1]

            answer_start = torch.argmax(start_index)
            answer_end = torch.argmax(end_index)

            answer = self.create_answer_text(tokens, answer_start, answer_end+2)

            answers.append(answer)

            c = start_index[0, answer_start].item() + end_index[0, answer_end].item()
            cs.append(c)

        maxC = max(cs)
        maxC_ind = cs.index(maxC)
        answer = answers[maxC_ind]
        return answer, maxC

    def train(self, documents, questions):
        df_dicts = {}
        summaries={}
        for question in questions:
            ids = []
            answers = []
            confs = []
            abstracts = []
            sent_ans=[]
            for document, abstract, id in notebook.tqdm(documents):
                answer, c = self.train_model_func(question, document)
                if answer!='':
                    ans_sent=self.get_answer_sentence(document, answer)
                    sent_ans.append(ans_sent)
                    ids.append(id)
                    answers.append(answer)
                    confs.append(c)
                    abstracts.append(abstract)
            
            
            question_df = pd.DataFrame({'Id': ids, 'abstract': abstracts, 'Answer': answers, 'answer_sent':sent_ans, 'conf': confs})
            question_df = question_df.sort_values(by=['conf'], ascending=False).reset_index(drop=True)
            summary=self.sum_model.get_summary(question_df, 100)
            df_dicts[question] = question_df
            summaries[question]=summary
            # df_lists.append(question_df)
        return df_dicts, summaries

In [5]:
Bert_config='/kaggle/input/bertuncaselargesquad'
use_cuda=torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')
token_p='bert-large-uncased-whole-word-masking-finetuned-squad'
Bart='bart-large-cnn'
#token_path='f{Bert_config}/vocab.txt'

In [38]:
question_model=QuestionAnswering(bert_model_path=Bert_config, token_model_path=token_p, device=device)

In [39]:
questions=['What is known about transmission,incubation,and environmental stability',
    'What do we know about COVID-19 risk factors',
    'What do we know about virus genetics, origin, and evolution',
    'What do we know about vaccines and therapeutics',
    'What has been published about medical care',
    'What do we know about non-pharmaceutical interventions',
    'What do we know about diagnostics and surveillance',
    'What has been published about ethical and social science considerations',
    'What has been published about information sharing and inter-sectoral collaboration']
# questions=[questions[0]]
# questions

In [ ]:
questin_m=['How does temperature and humidity affect the tramsmission',
          'What do we know about natural history, transmission, and diagnostics',
          'How does smoking affect',
          'How does pregnancy affect patients',
          'Can animals transmit',
          'What animal did 2019-nCoV come from',
          'What drugs or therapies are being investigated',
          'What antiviral drug clinical trials have been completed',
          'How is artificial intelligence being used in real time health delivery',
          'What adjunctive or supportive methods can help patients',
          'What are most important barriers to compliance',
          'What is being done to increase testing capacity or throughput',
          'What collaborations are happening within the research community',
           'What factors contribute to rumors and misinformation',
           'How do pandemics affect the physical and/or psychological health of doctors and nurses']

In [21]:
ddir='/kaggle/input/articles/AllArticles.csv'

df=joblib.load(ddir)
df=df[df['is_covid19']==True]
#df=df[:1000]
df.shape
#texts=df.abstract.values
texts=df.body_text.values
ids=df.Id.values
absts=df.abstract.values

del df

documents=[(texts[i],absts[i], ids[i]) for i in range(len(texts))]
del texts
del ids
del absts

In [32]:
import gc
gc.collect()

10275

In [ ]:
df.head()

In [40]:
dicts,sums=question_model.train(documents,questions)

In [41]:
sums['What is known about transmission,incubation,and environmental stability']

' The main symptoms were fever, cough, dyspnea, mygia, fatigue, and radiographic evidence of pneumonia. It still remains unclear about the origin, transmission mode and incubation period of the sars-cov-2. The disease is highly transmissible mostly through coughing and sneezing dropls, contaminated surfaces and aerosols. The major source of infection is the patients with covid-19. Covid-19 is thought to be transmitted through dropls, close contact, aerosol and maybe fec-ortransmission,  and patients in the incubation period can transmit the virus to other persons. There are significant knowledge gaps in the epidemiology, transmission dynamics, investigation tools, and management of covid- 19. The 2019 novel coronavirus sars-cov-2 has been dected in the patients in the wuhan city of china. The mode of transmission is person to person via respiratory dropls and contact. The number of patients with covid-19 has rapidly increased, with nearly 80,000 reported cases. Covid-19 is spread by hu

In [ ]:
len(dicts)

In [42]:
dicts['What is known about transmission,incubation,and environmental stability'].head(10)

,Id,abstract,Answer,answer_sent,conf
0,ed6e34937c0ad11574f619e32adab283699082c9,Background: The outbreak of a novel coronaviru...,. human-to-human transmission of ncip has been...,"the main symptoms were fever, cough, dyspnea,...",10.030523
1,a7d8b659f420dbf2fbe53fd75258ee42e58f24ea,A cluster of pneumonia associated with the SAR...,", it still remains unclear about the origin, t...","furthermore , it still remains unclear about ...",9.898027
2,4cd306cff8ce22bd079af8ad5750aae567d78126,JournPre-proof J o u r n a l P r e -p r o o f ...,the transmission dynamics is not fully understood,the transmission dynamics is not fully under...,9.214188
3,4c58456723e46f7551d3e1ee8e9a1cc1eadf8d7c,,but serious questions regarding the causes or ...,there have been more than 50 research papers ...,9.097869
4,dac1b1607ae72b9509ab26367e0d55016e8132a8,Objective] Anyze the occurrence of novel coron...,. there is clear the route of transmission of ...,. there is clear the route of transmission o...,8.856227
5,2d99df39f474b652e4c61e1d3e166f43020c3524,Objective: To survey fertility patients' agree...,", little is known on the impact of covid-19 on...","at this time , little is known on the impact ...",8.735323
6,4c3cf192b70369b7e40dab26f1ff79da2e431b02,,. airborne transmission remains unclear,this disease is highly transmissible mostly t...,8.607716
7,a9ff060e7763044e4047398f76b6a3cca8b3fbf0, The age and underlying diseases were the mos...,the person-to-person transmission in hospitand...,it is reported that the person-to-person tra...,8.580153
8,3c94f582610453b6c1389c3a5d6ad57bcf0ba033,confirmed and 1368 patients have died from the...,the person-to-person transmission in hospitand...,it is reported that the person-to-person tra...,8.315717
9,9b3c8ae64057fb199a1770ca46929ce031c34b56,,and transmission seems to occur mainly through...,although sufficient information is not yavail...,8.225224


In [43]:
joblib.dump(dicts, '/kaggle/working/Newdictionaries')

['/kaggle/working/Newdictionaries']

In [44]:
joblib.dump(sums, '/kaggle/working/Newsummaries')

['/kaggle/working/Newsummaries']

In [ ]:
ndicts,nsums=question_model.train(documents,questin_m)

In [ ]:
pd.set_option('display.max_colwidth',100)

In [ ]:
dicts['What is known about transmissio,incubation,and environmental stability?'].head(10)

In [ ]:
doc=[t for t,a,i in documents if i=='cb3974f33a1d54369d630e62052d603148bb12e7']

In [ ]:
doc[0]